In [1]:
%matplotlib inline
import torch 
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
import vulcanai

SyntaxError: invalid syntax (__init__.py, line 1)

In [ ]:
from vulcanai import mnist_loader
(train_images, train_labels, test_images, test_labels) = mnist_loader.load_fashion_mnist()

In [ ]:
from utils import get_one_hot
train_labels = get_one_hot(train_labels)
test_labels = get_one_hot(test_labels)

In [ ]:
train_images = train_images.reshape((-1, 28, 28, 1)).astype(np.float32)
test_images = test_images.reshape((-1, 28, 28, 1)).astype(np.float32)
train_labels = train_labels.astype(np.int64)
test_labels = test_labels.astype(np.int64)
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class TMPDataset(Dataset): #Source: https://discuss.pytorch.org/t/problems-with-pytorch-mlp-when-training-the-mnist-dataset-retrieved-from-keras/12918

    def __init__(self, a, b, transform=None):
        self.x = a
        self.y = b
        
        self.transform = transform;

    def __getitem__(self, idx):
        item = self.x[idx];
        label = self.y[idx];
        
        if self.transform:
            item = self.transform(item);
        
        return (item, label);

    def __len__(self):
        return len(self.y);
train_dataset = TMPDataset(test_images, test_labels)
test_dataset = TMPDataset(train_images, train_labels)
    
test_loader = DataLoader(train_dataset, num_workers=1, batch_size=10000)
train_loader = DataLoader(test_dataset, shuffle=True, batch_size=1000)



In [ ]:
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt',
              7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'};

In [ ]:
#source: https://medium.com/ml2vec/intro-to-pytorch-with-image-classification-on-a-fashion-clothes-dataset-e589682df0c5
fig = plt.figure(figsize=(8,8));
columns = 4;
rows = 5;

for i in range(1, columns*rows +1):
    img_xy = np.random.randint(len(train_dataset));
    img = train_dataset[img_xy][0][:,:,0]
    fig.add_subplot(rows, columns, i)
    plt.title(labels_map[np.argmax(train_dataset[img_xy][1])])
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
network_conv_config = {
    'mode': 'conv',
    'filters': [16, 32],
    'filter_size': [[5, 5], [5, 5]],
    'stride': [[1, 1], [1, 1]],
    'pool': {
        'mode': 'average_exc_pad',
        'stride': [[2, 2], [2, 2]]
    }
}
network_dense_config = {
    'mode': 'dense',
    'units': [512],
    'dropouts': [0.3],
}

In [ ]:
import net

In [ ]:
train_images.shape

In [ ]:
#input_var = Variable(train_images.shape, requires_grad=True)
#y = Variable(train_labels.shape, requires_grad=True)
conv_net= net.Network(
    name='conv_test',
    dimensions= train_images.shape,
    input_var=None,
    y=None,
    config=network_conv_config,
    input_network=None,
    num_classes=None)

In [ ]:
dense_net = net.Network(
    name='1_dense',
    dimensions=(None, int(train_images.shape[1])),
    input_var=0,
    y=0,
    config=network_dense_config,
    input_network={'network': conv_net, 'layer': 4, 'get_params': True},
    num_classes=10,
    activation='rectify',
    pred_activation='softmax'
)

In [ ]:
conv_net.network

In [ ]:
dense_net.network

In [ ]:
Sequential(
  (conv_test_input): Linear(in_features=1, out_features=1, bias=True)
  (conv_test_conv2D_0): Sequential(
    (0): Conv2d(1, 16, kernel_size=[5, 5], stride=[1, 1])
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  )
  (conv_test_conv2D_1): Sequential(
    (0): Conv2d(16, 32, kernel_size=[5, 5], stride=[1, 1])
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten()
  (1_dense_dense_0): Sequential(
    (0): Linear(in_features=32, out_features=512, bias=True)
    (1): Dropout(p=0.3)
  )
  (classification_layer): Sequential(
    (0): Linear(in_features=512, out_features=10, bias=True)
    (1): Softmax()
  )
)

In [ ]:

mods = list(dense_net.network.modules())
for i in range(1,len(mods)):
    m = mods[i]
    p = list(m.parameters())
    sizes = []
    print(m)
    for j in range(len(p)):
        size = np.array(p[j].size())
        sizes.append(size)
        print(size)

total_bits = 0
for i in range(len(sizes)):
    s = sizes[i]
    bits = np.prod(np.array(s))*bits
    print(bits)
    total_bits += bits

In [ ]:
for l_name, l in dense_net.network.named_children():
    if isinstance(l, torch.nn.Sequential):
        for subl_name, subl in l.named_children():
            for param in subl.parameters():
                print(l_name, subl_name, param.size(0))
    else:
        for param in l.parameters():
            print(l_name, param.size(0))


In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = dense_net.network
        
    def forward(self, x):
        out = self.layer(x)
        return out

In [ ]:
cnn = CNN()
#loss function and optimizer
criterion = torch.nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01);

In [ ]:
losses = [];
for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data[0]);
        
        if (i+1) % 100 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

In [ ]:
seq = 0
for l_name, l in model.network.named_children():
    if isinstance(l, torch.nn.Sequential):
        for subl_name, subl in l.named_children():
            for param in subl.parameters():
                print(subl, param.size(0))

    else:       
        for param in l.parameters():
            print(l, param.size(0))

In [ ]:
for m_name, m in model.named_children():
    model2.add_module(m_name, m)
model2

In [ ]:
model.layers

In [ ]:
for idx, m in enumerate(model2.modules()):
    print(type(idx), '->', type(m))
    print(idx, '->', m)
    

In [ ]:
model[0]

In [ ]:
model3 = torch.nn.Sequential(, model[0])

In [ ]:
model3